## 章节前言
自编码器是能够在无监督（即，训练集是未标记）的情况下学习输入数据的紧密表征（叫做潜在表征或编码）的人工神经网络。这些编码通常具有比输入数据低得多的维度，使得自编码器对降维有用（参见第 8 章）。自编码器还可以作为强大的特征检测器，它们可以用于无监督的深度神经网络预训练（正如我们在第 11 章中讨论过的）。最后，一些自编码器是生成式模型：他们能够随机生成与训练数据非常相似的新数据。例如，您可以在脸图片上训练自编码器，然后可以生成新脸。但是生成出来的图片通常是模糊且不够真实。

相反，用对抗生成网络（GAN）生成的人脸可以非常逼真，甚至让人认为他们是真实存在的人。你可以去这个网址，这是用 StyleGAN 生成的人脸，自己判断一下（还可以去这里，看看 GAN 生成的卧室图片），GAN 现在广泛用于超清图片涂色，图片编辑，将草图变为照片，增强数据集，生成其它类型的数据（比如文本、音频、时间序列），找出其它模型的缺点并强化，等等。

自编码器和 GAN 都是无监督的，都可以学习紧密表征，都可以用作生成模型，有许多相似的应用，但原理非常不同：

自编码器是通过学习，将输入复制到输出。听起来很简单，但内部结构会使其相当困难。例如，你可以限制潜在表征的大小，或者可以给输入添加噪音，训练模型恢复原始输入。这些限制阻止自编码器直接将输入复制到输出，可以强迫模型学习数据的高效表征。总而言之，编码是自编码器在一些限制下学习恒等函数的副产品。

GAN 包括两个神经网络：一个生成器尝试生成和训练数据相似的数据，一个判别器来区分真实数据和假数据。特别之处在于，生成器和判别器在训练过程中彼此竞争：生成器就像一个制造伪钞的罪犯，而判别器就像警察一样，要把真钱挑出来。对抗训练（训练竞争神经网络），被认为是近几年的一大进展。在 2016 年，Yann LeCun 甚至说 GAN 是过去 10 年机器学习领域最有趣的发明。

本章中，我们先探究自编码器的工作原理，如何做降维、特征提取、无监督预训练将、如何用作生成式模型。然后过渡到 GAN。先用 GAN 生成假图片，可以看到训练很困难。会讨论对抗训练的主要难点，以及一些解决方法。先从自编码器开始。

## 有效的数据表征
以下哪一个数字序列更容易记忆？

40, 27, 25, 36, 81, 57, 10, 73, 19, 68
50, 48, 46, 44, 42, 40, 38, 36, 34, 32, 30, 28, 26, 24, 22, 20, 18, 16, 14

乍一看，第一个序列似乎应该更容易，因为它要短得多。 但是，如果仔细观察第二个序列，就会发现它是从 50 到 14 的偶数。一旦你注意到这个规律，第二个序列比第一个更容易记忆，因为你只需要记住规律就成，开始的数字和结尾的数字。请注意，如果您可以快速轻松地记住非常长的序列，则不会在意第二个序列中存在的规律。 只要记住每一个数字，就够了。 事实上，很难记住长序列，因此识别规律非常有用，并且希望能够澄清为什么在训练过程中限制自编码器会促使它发现并利用数据中的规律。

记忆、感知和模式匹配之间的关系在 20 世纪 70 年代早期由 William Chase 和 Herbert Simon 研究。 他们观察到，专业棋手能够通过观看棋盘 5 秒钟就能记住所有棋子的位置，这是大多数人认为不可能完成的任务。 然而，只有当这些棋子被放置在现实位置（来自实际比赛）时才是这种情况，而不是随机放置棋子。 国际象棋专业棋手没有比你更好的记忆，他们只是更容易看到国际象棋的规律，这要归功于他们的比赛经验。 观察规律有助于他们有效地存储信息。

就像这个记忆实验中的象棋棋手一样，一个自编码器会查看输入信息，将它们转换为高效的潜在表征，然后输出一些（希望）看起来非常接近输入的东西。 自编码器总是由两部分组成：将输入转换为潜在表征的编码器（或识别网络），然后是将潜在表征转换为输出的解码器（或生成网络）（见图 17-1）。

<img src='
https://hands1ml.apachecn.org/img/133f15a30a9de1a24f0a24873dd09824.png
'>
<br>